In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.corpus import stopwords
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scraped_Car_Review_lotus.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scraped_Car_Review_porsche.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scraped_Car_Review_hummer.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scraped_Car_Review_mazda.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scrapped_Car_Reviews_Toyota.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scraped_Car_Review_lexus.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scraped_Car_Review_subaru.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scrapped_Car_Reviews_Acura.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scrapped_Car_Reviews_Audi.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scrapped_Car_Reviews_Honda.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scraped_Car_Review_jeep.csv
/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scrapped_Car_Reviews_B

# Aspect Base Sentiment Analysis

In [16]:
maserati = pd.read_csv('/kaggle/input/edmundsconsumer-car-ratings-and-reviews/Scraped_Car_Review_maserati.csv')
maserati.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,0,on 01/01/05 19:28 PM (PST),Jeff,2004 Maserati Spyder Convertible Cambiocorsa 2...,Maserati Thrill Ride,I bought my Maserati Spyder Cambiocorsa \rin ...,4.875
1,1,on 12/05/04 06:07 AM (PST),Francie,2004 Maserati Spyder Convertible Cambiocorsa 2...,My second Spyder,I got the very first Maserati Spyder delivere...,4.750
2,2,on 12/04/04 15:28 PM (PST),West8,2004 Maserati Spyder Convertible GT 2dr Conver...,Fast & Fun,After owning several comparable cars in the p...,4.375
3,3,on 10/28/04 00:00 AM (PDT),KLAMIA2001,2004 Maserati Spyder Convertible Cambiocorsa 2...,ITALIANS RULE,"THIS IS MY THIRD MASERATI, A 2002, 2003 \r& A...",5.000
4,4,on 09/22/04 00:00 AM (PDT),d blair,2004 Maserati Spyder Convertible Cambiocorsa 2...,Midlife Crisis?? I don't think so!,Ok so I am a 55 year old man who has,NaN


We combine the review title and its body, as both can contain information about sentiment

In [17]:
maserati["Review"] = maserati["Review_Title"] + ". " + maserati["Review"]

# Dependancy Parsing
* One of the core feature required for aspect extraction.
* **Dependency Parsing**(DP) refers to examining the dependencies between the words of a sentence to analyze its grammatical structure. Based on this, a sentence is broken into several components. The mechanism is based on the concept that there is a direct link between every linguistic unit of a sentence.
![Dependancy Parsing](https://editor.analyticsvidhya.com/uploads/32408Screenshot%20(128).png)

In [18]:
import spacy
from tqdm import tqdm
#We use spacy for dependacy parsing
#tqdm is used to provide progress bars
nlp = spacy.load('en_core_web_lg')

# Dependancy Representation
* Details can be found [here](https://downloads.cs.stanford.edu/nlp/software/dependencies_manual.pdf)
* **AMOD** - Adjective modifier, any adjective that serves ot modify the meaning of a Noun.
* **ADVMOD** - Adverb modifier, an adverb or adverb-headed phrase that serves to modify the meaning of the word.
* **XCOMP** - Open causal complement, An open clausal complement (xcomp) of a verb or an adjective is a predicative or clausal complement without its own subject.
* **NEG**

In [19]:
txt = 'She looks very beautiful, with a great scenary not behind her.'
doc = nlp(txt)
for token in doc:
    print(token.text, token.lemma_, token.pos_)
spacy.displacy.render(doc,style='dep',jupyter=True)

She she PRON
looks look VERB
very very ADV
beautiful beautiful ADJ
, , PUNCT
with with ADP
a a DET
great great ADJ
scenary scenary ADJ
not not PART
behind behind ADP
her she PRON
. . PUNCT


In [20]:
#We use a list of competitor car manufactuers to avoid miscalssifications.
comp = ['Chevy','chevy','Dodge','dodge','Ford','ford','Ferrari','ferrari','Nissan','nissan','Honda','honda','Chevrolet','chevrolet','Volkswagen','volkswagen','benz','Benz','Mercedes','mercedes','subaru','Subaru','VW']

In [21]:
s = "This is a test. * -324 [],"
re.sub(r'[^A-Za-z0-9. ]+', '', s)
maserati["Review"][0]
lines = re.sub(r'[^A-Za-z0-9. ]+', '', str(maserati['Review'][0])).split(".")
for line in lines:
    print(line)

Maserati Thrill Ride
  I bought my Maserati Spyder Cambiocorsa in May of 2004
  The car is a light metallic blue with a light tan and navy interior
  The car is fully optioned
  The interior is the best looking interior of any car I have ever been in
  Leather is everywhere and it is of the highest quality
  While ferrari does own maserati they are very different cars
 This car is a GT car smoother ride and without the wild looks of a ferrari but it still gets tons of attention
 It is in the same price range as the SL500 but much rarer and faster
  Do not get the nav system it is almost useless  barely showing any information
  The paddle shifters are amazing though the 6 speed manual is very smooth too



In [22]:
aspect_terms = []
compound_all = []
easpect_terms = []
comp_compound_all = []
enemy = []
for x in tqdm(range(len(maserati['Review']))):
    aspects = {"amod":[], "advmod":[], "compound":[], "xcomp":[], "neg":[]}
    aspects_comp = {"amod":[], "advmod":[], "compound":[], "xcomp":[], "neg":[]}
    enemlist = []
    if len(str(maserati['Review'][x])) != 0:
        lines = re.sub(r'[^A-Za-z0-9. ]+', '', str(maserati['Review'][x])).split(".")
        for line in lines:
            line = line.strip()
            enem_list = []
            for eny in comp:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.is_stop: continue
                    if token.pos_ == 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                aspects["compound"].append((j.text+' '+token.text,token.text))
                            if j.dep_ == 'amod' and j.pos_ == 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                aspects["amod"].append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ == 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        aspects["advmod"].append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    aspects["advmod"].remove(str1)
                    if token.pos_ == 'VERB':
                        for j in token.lefts:
                            if j.dep_ == 'advmod' and j.pos_ == 'ADV':
                                aspects["advmod"].append(j.text+' '+token.text)
                            if j.dep_ == 'neg' and j.pos_ == 'ADV':
                                aspects["neg"].append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ == 'advmod'and j.pos_ == 'ADV':
                                aspects["advmod"].append(token.text+' '+j.text)
                    if token.pos_ == 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ == 'xcomp' and h.dep_ != 'neg':
                                for k in j.lefts:
                                    if k.dep_ == 'aux':
                                        aspects["xcomp"].append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ == 'xcomp' and h.dep_ == 'neg':
                                if k.dep_ == 'aux':
                                        aspects["neg"].append(h.text +' '+token.text+' '+k.text+' '+j.text)
            
            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ == 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                aspects_comp["compound"].append((j.text+' '+token.text,token.text))
                            if j.dep_ == 'amod' and j.pos_ == 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                aspects_comp["amod"].append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ == 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        aspects_comp["advmod"].append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    aspects_comp["amod"].remove(str1)
                    if token.pos_ == 'VERB':
                        for j in token.lefts:
                            if j.dep_ == 'advmod' and j.pos_ == 'ADV':
                                aspects_comp["advmod"].append(j.text+' '+token.text)
                            if j.dep_ == 'neg' and j.pos_ == 'ADV':
                                aspects_comp["neg"].append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ == 'advmod'and j.pos_ == 'ADV':
                                aspects_comp["advmod"].append(token.text+' '+j.text)
                    if token.pos_ == 'ADJ':
                        for j in token.rights:
                            if j.dep_ == 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ == 'aux':
                                        aspects_comp["xcomp"].append(token.text+' '+k.text+' '+j.text)
        pairs = set()
        #epairs = set()
        for key in aspects.keys():
            if key == "compound": continue
            pairs = pairs.union(set(aspects[key]))
            #epairs = epairs.union(set(aspects_comp[key]))
        pairs = list(pairs)
        #epairs = list(epairs)
        for i in range(len(pairs)):
            for comp in aspects["compound"]:
                mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                if mtch is not None:
                    pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        '''for i in range(len(epairs)):
            for comp in aspects_comp["compound"]:
                mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                if mtch is not None:
                    epairs[i] = epairs[i].replace(mtch.group(),comp[0])'''
            
    aspect_terms.append(pairs)
    #compound_all.append(aspects["compound"])
    #easpect_terms.append(epairs)
    #comp_compound_all.append(aspects_comp["compound"])
    #enemy.append(enemlist)
#maserati['compound_nouns'] = compound_all
maserati['aspect_keywords'] = aspect_terms
#maserati['competition'] = enemy
#maserati['competition_comp_nouns'] = comp_compound_all
#maserati['competition_aspects'] = easpect_terms
maserati.head()

100%|██████████| 302/302 [00:18<00:00, 16.09it/s]


,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,aspect_keywords
0,0,on 01/01/05 19:28 PM (PST),Jeff,2004 Maserati Spyder Convertible Cambiocorsa 2...,Maserati Thrill Ride,Maserati Thrill Ride. I bought my Maserati Sp...,4.875,"[metallic blue, highest quality, light tan, sa..."
1,1,on 12/05/04 06:07 AM (PST),Francie,2004 Maserati Spyder Convertible Cambiocorsa 2...,My second Spyder,My second Spyder. I got the very first Masera...,4.750,"[second Spyder, new model, black interior, tan..."
2,2,on 12/04/04 15:28 PM (PST),West8,2004 Maserati Spyder Convertible GT 2dr Conver...,Fast & Fun,Fast & Fun. After owning several comparable c...,4.375,"[Now spends, only problem, comparable cars, bl..."
3,3,on 10/28/04 00:00 AM (PDT),KLAMIA2001,2004 Maserati Spyder Convertible Cambiocorsa 2...,ITALIANS RULE,"ITALIANS RULE. THIS IS MY THIRD MASERATI, A 2...",5.000,[MORE CARISMATIC CAR]
4,4,on 09/22/04 00:00 AM (PDT),d blair,2004 Maserati Spyder Convertible Cambiocorsa 2...,Midlife Crisis?? I don't think so!,Midlife Crisis?? I don't think so!. Ok so I a...,NaN,"[think so, old man]"


In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [24]:
import operator
sentiment = []
for i in range(len(maserati)):
    score_dict={'pos':0,'neg':0,'neu':0}
    if len(maserati['aspect_keywords'][i])!=0: 
        for aspects in maserati['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
            #score_dict['neu'] += sent['neu']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
maserati['sentiment'] = sentiment
maserati.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,aspect_keywords,sentiment
0,0,on 01/01/05 19:28 PM (PST),Jeff,2004 Maserati Spyder Convertible Cambiocorsa 2...,Maserati Thrill Ride,Maserati Thrill Ride. I bought my Maserati Sp...,4.875,"[metallic blue, highest quality, light tan, sa...",pos
1,1,on 12/05/04 06:07 AM (PST),Francie,2004 Maserati Spyder Convertible Cambiocorsa 2...,My second Spyder,My second Spyder. I got the very first Masera...,4.750,"[second Spyder, new model, black interior, tan...",pos
2,2,on 12/04/04 15:28 PM (PST),West8,2004 Maserati Spyder Convertible GT 2dr Conver...,Fast & Fun,Fast & Fun. After owning several comparable c...,4.375,"[Now spends, only problem, comparable cars, bl...",neg
3,3,on 10/28/04 00:00 AM (PDT),KLAMIA2001,2004 Maserati Spyder Convertible Cambiocorsa 2...,ITALIANS RULE,"ITALIANS RULE. THIS IS MY THIRD MASERATI, A 2...",5.000,[MORE CARISMATIC CAR],pos
4,4,on 09/22/04 00:00 AM (PDT),d blair,2004 Maserati Spyder Convertible Cambiocorsa 2...,Midlife Crisis?? I don't think so!,Midlife Crisis?? I don't think so!. Ok so I a...,NaN,"[think so, old man]",pos


In [25]:
int_sent = []
for sent in maserati['sentiment']:
    if sent == 'NaN':
        int_sent.append('NaN')
    elif sent == 'pos':
        int_sent.append('1')
    else:
        int_sent.append('0')
maserati['int_sent'] = int_sent
maserati.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,aspect_keywords,sentiment,int_sent
0,0,on 01/01/05 19:28 PM (PST),Jeff,2004 Maserati Spyder Convertible Cambiocorsa 2...,Maserati Thrill Ride,Maserati Thrill Ride. I bought my Maserati Sp...,4.875,"[metallic blue, highest quality, light tan, sa...",pos,1
1,1,on 12/05/04 06:07 AM (PST),Francie,2004 Maserati Spyder Convertible Cambiocorsa 2...,My second Spyder,My second Spyder. I got the very first Masera...,4.750,"[second Spyder, new model, black interior, tan...",pos,1
2,2,on 12/04/04 15:28 PM (PST),West8,2004 Maserati Spyder Convertible GT 2dr Conver...,Fast & Fun,Fast & Fun. After owning several comparable c...,4.375,"[Now spends, only problem, comparable cars, bl...",neg,0
3,3,on 10/28/04 00:00 AM (PDT),KLAMIA2001,2004 Maserati Spyder Convertible Cambiocorsa 2...,ITALIANS RULE,"ITALIANS RULE. THIS IS MY THIRD MASERATI, A 2...",5.000,[MORE CARISMATIC CAR],pos,1
4,4,on 09/22/04 00:00 AM (PDT),d blair,2004 Maserati Spyder Convertible Cambiocorsa 2...,Midlife Crisis?? I don't think so!,Midlife Crisis?? I don't think so!. Ok so I a...,NaN,"[think so, old man]",pos,1


In [26]:
import math
pos = []
for i in range(len(maserati)):
    if not math.isnan(maserati['Rating'][i]):
        if int(maserati['Rating'][i])>3:
            pos.append('1')
        else:
            pos.append('0')
    else:
        pos.append('0')
maserati['Positive Review'] = pos
maserati.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,aspect_keywords,sentiment,int_sent,Positive Review
0,0,on 01/01/05 19:28 PM (PST),Jeff,2004 Maserati Spyder Convertible Cambiocorsa 2...,Maserati Thrill Ride,Maserati Thrill Ride. I bought my Maserati Sp...,4.875,"[metallic blue, highest quality, light tan, sa...",pos,1,1
1,1,on 12/05/04 06:07 AM (PST),Francie,2004 Maserati Spyder Convertible Cambiocorsa 2...,My second Spyder,My second Spyder. I got the very first Masera...,4.750,"[second Spyder, new model, black interior, tan...",pos,1,1
2,2,on 12/04/04 15:28 PM (PST),West8,2004 Maserati Spyder Convertible GT 2dr Conver...,Fast & Fun,Fast & Fun. After owning several comparable c...,4.375,"[Now spends, only problem, comparable cars, bl...",neg,0,1
3,3,on 10/28/04 00:00 AM (PDT),KLAMIA2001,2004 Maserati Spyder Convertible Cambiocorsa 2...,ITALIANS RULE,"ITALIANS RULE. THIS IS MY THIRD MASERATI, A 2...",5.000,[MORE CARISMATIC CAR],pos,1,1
4,4,on 09/22/04 00:00 AM (PDT),d blair,2004 Maserati Spyder Convertible Cambiocorsa 2...,Midlife Crisis?? I don't think so!,Midlife Crisis?? I don't think so!. Ok so I a...,NaN,"[think so, old man]",pos,1,0


In [27]:
d = {'sent':maserati['Positive Review'],'sent_pred':maserati['int_sent']}
metric_df = pd.DataFrame(data=d)
metric_df = metric_df[metric_df.sent_pred != 'NaN']
metric_df.head()

,sent,sent_pred
0,1,1
1,1,1
2,1,0
3,1,1
4,0,1


In [28]:
from sklearn.metrics import accuracy_score,auc,f1_score,recall_score,precision_score
print('accuracy')
print(accuracy_score(metric_df.sent, metric_df.sent_pred))
print('f1 score')
print(f1_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('recall')
print(recall_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('precision')
print(precision_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))

accuracy
0.8130434782608695
f1 score
0.8877284595300261
recall
0.9139784946236559
precision
0.8629441624365483
